In [7]:
pip install openpyxl

     |████████████████████████████████| 243 kB 1.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
import numpy as np
import pandas as pd
import re

### TODO
- Step 1: Create name list from 'raw data file'
- Step 2: Filter secur_name from 'top 5 file'
- Step 3: Create mapping file
- Step 4: Generate result

## Step 1: Create name list from 'raw data file'

- save to file 'raw-data-name.csv' for checking error
- row format should be 'Name|col_index'

In [139]:
# Read excel to dataframe
src_filename="data_source/Single Stock Total Return Raw Data.xlsm"
sheet_name='Sheet1'
dest_filename='check_file/raw-data-name.csv'

df = pd.read_excel(open(src_filename, 'rb'),sheet_name=sheet_name)
df.head()

,Start,2011-12-30 00:00:00,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 608,Unnamed: 609,Unnamed: 610,Unnamed: 611,Unnamed: 612,Unnamed: 613,Unnamed: 614,Unnamed: 615,Unnamed: 616,Unnamed: 617
0,End,2021-04-20 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Frequency,D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Name,THAI ENERGY STORAGE TECHNOLOGY - TOT RETURN IND,SEVEN UTILITIES AND POWER ORS - TOT RETURN IND,AREEYA PROPERTY - TOT RETURN IND,ASIA AVIATION - TOT RETURN IND,AMATA B GRIMM PWPL.IFCF. - TOT RETURN IND,ADVANCED CONNECTION CORPORATION - TOT RETURN IND,ABSOLUTE CLEAN ENERGY - TOT RETURN IND,AUTOCORP HOLDING - TOT RETURN IND,ADVANCED INFO SER. - TOT RETURN IND,...,WHA PRUM.GW.FH.&. LH. REIT - TOT RETURN IND,WHA UTILITIES AND POWER - TOT RETURN IND,WICE LOGISTICS - TOT RETURN IND,WIIK - TOT RETURN IND,WYNCOAST INDUSTRIAL PARK - TOT RETURN IND,WORKPOINT ENTERTAINMENT - TOT RETURN IND,WP ENERGY - TOT RETURN IND,WATTANAPAT HOSPITAL TRANG - TOT RETURN IND,ZEN CORPORATION - TOT RETURN IND,SEAMICO CAPITAL - TOT RETURN IND
3,Code,9828W5(RI),15376W(RI),28751E(RI),87190C(RI),90541H(RI),316790(RI),9527VT(RI),9457XP(RI),314926(RI),...,9237YP(RI),9063AG(RI),9827VY(RI),865372(RI),540250(RI),29581C(RI),9216HC(RI),9166LF(RI),9404PC(RI),141012(RI)
4,2011-12-30 00:00:00,NaN,54.21,78.39,NaN,NaN,73.27,NaN,NaN,7878.38,...,NaN,NaN,NaN,75.71,0.15,145.23,NaN,NaN,NaN,21.83


In [140]:
# select row 2
df = df[2:3]

# change header to index
df.columns = range(df.shape[1])
df = df.transpose()
df['new_index'] = df.index

# remove "- TOT RETURN IND"
df.replace(to_replace=r'- TOT RETURN IND$', value='', regex=True, inplace=True)

# save file
df.to_csv(dest_filename, header=False, index=False, sep='|')
df.head()

,2,new_index
0,Name,0
1,THAI ENERGY STORAGE TECHNOLOGY,1
2,SEVEN UTILITIES AND POWER ORS,2
3,AREEYA PROPERTY,3
4,ASIA AVIATION,4


## Step 2: Filter secur_name from 'top 5 file'

- secur_abbr_name field must not be number
- remove duplicate
- save to file 'top5-name.csv' for checking error

In [141]:
# Read excel to dataframe
src_filename="data_source/top5_201306.xlsx"
sheet_name='Sheet1'
dest_filename='check_file/top5-name.csv'

df = pd.read_excel(open(src_filename, 'rb'),sheet_name=sheet_name)
print(df.shape)
df.head()

(778, 7)


,Unnamed: 0,proj_id,assetseq,secur_name,secur_abbr_name,period,secur_invest_size
0,0,M0008_2537,1,ADVANCED INFO SERVICE PUBLIC COMPANY LIMITED,ADVANC,201306,9.48
1,1,M0008_2537,2,THE SIAM CEMENT PUBLIC COMPANY LIMITED,SCC,201306,5.56
2,2,M0008_2537,3,KASIKORNBANK PUBLIC COMPANY LIMITED,KBANK,201306,5.56
3,3,M0008_2537,4,PTT PUBLIC COMPANY LIMITED,PTT,201306,5.51
4,4,M0008_2537,5,THE SIAM COMMERCIAL BANK PUBLIC COMPANY LIMITED,SCB,201306,4.97


In [142]:
# Filter abbr_name with number + '-'
filter_name = df.loc[~df['secur_abbr_name'].str.match("^[0-9/-]*$")]
# Remove 'PUBLIC COMPANY LIMITED'
filter_name.replace(to_replace=r'PUBLIC COMPANY LIMITED$', value='', regex=True, inplace=True)
filter_name = filter_name.sort_values(by=['secur_name'])
print(filter_name.shape)
filter_name.head()

(761, 7)


/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:4524: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


,Unnamed: 0,proj_id,assetseq,secur_name,secur_abbr_name,period,secur_invest_size
0,0,M0008_2537,1,ADVANCED INFO SERVICE,ADVANC,201306,9.48
390,390,M0288_2554,1,ADVANCED INFO SERVICE,ADVANC,201306,6.88
406,406,M0423_2549,2,ADVANCED INFO SERVICE,ADVANC,201306,6.88
412,412,M0526_2546,3,ADVANCED INFO SERVICE,ADVANC,201306,5.96
417,417,M0583_2547,3,ADVANCED INFO SERVICE,ADVANC,201306,7.84


In [143]:
# Drop duplicate secur_name name
df = filter_name['secur_name']
df = df.drop_duplicates()
df.to_csv(dest_filename, header=False, index=False)
print(df.shape)
df.head()

(84,)


0      ADVANCED INFO SERVICE 
182     AIRPORTS OF THAILAND 
569        AMATA CORPORATION 
424            ASIA AVIATION 
467     ASIA SERMKIJ LEASING 
Name: secur_name, dtype: object

## Step 3: Create mapping file
- mapping between 'top5-name.csv' and 'raw-data-name.csv'
- using longest matching substring
- save to file 'mapping.csv' for checking error
- row format should be 'top5-name,[Name|col_index,Name|col_index,Name|col_index]'
- **Need manual checking file and confirm result


In [94]:
def LCSubStr(s, t, n, m):
   
    # Create DP table
    dp = [[0 for i in range(m + 1)] for j in range(2)]
    res = 0
     
    for i in range(n + 1):
        for j in range(m + 1):
            if(s[i - 1] == t[j - 1]):
                dp[i % 2][j] = dp[(i - 1) % 2][j - 1] + 1
                if(dp[i % 2][j] > res):
                    res = dp[i % 2][j]
            else:
                dp[i % 2][j] = 0
    return res
 
# Driver Code
X = "OldSite:GeeksforGeeks.org"
Y = "NewSite:GeeksQuiz.com"
m = len(X)
n = len(Y)
 
# Function call
print(LCSubStr(X,Y,m,n))

10


In [146]:
# Create mapping top5 --> raw
top5_filename="check_file/top5-name.csv"
raw_filename="check_file/raw-data-name.csv"

raw_name_list = []
with open(raw_filename, 'r') as file:
    for line in file: 
        line = line.strip()
        raw_name_list.append(line)

print("---------------")
print(raw_name_list[0:5])
print("---------------")

result = dict()
with open(top5_filename, 'r') as file:
    for i, line in enumerate(file):
        max_matching = 0
        mapping_name = []

        line = line.strip()
#         print(i, line)

        for raw in raw_name_list:
            m = LCSubStr(line,raw,len(line),len(raw))
            if(m == 0):
                continue
            if(m > max_matching):
                max_matching = m
                mapping_name.clear()
                mapping_name.append(raw)
#                 mapping_name.append(raw + "(" + str(m) + ")")
            elif(m == max_matching):
                mapping_name.append(raw)
#                 mapping_name.append(raw + "(" + str(m) + ")")
        
        result[line] = mapping_name.copy()

des_filename="check_file/mapping.csv"
with open(des_filename, 'w+') as out_file:
    for k,v in result.items():
        if len(list(v)) > 1:
            out_file.write("----- [TODO] -----")
        values = ','.join([str(elem) for elem in v])
        out_file.write(k+","+values+"\n")

---------------
['Name|0', 'THAI ENERGY STORAGE TECHNOLOGY |1', 'SEVEN UTILITIES AND POWER ORS |2', 'AREEYA PROPERTY |3', 'ASIA AVIATION |4']
---------------


## Step 4: Generate result
- using file 'raw data file' and file from step 3 to generate result file

In [155]:
def excel_column_name(n):
    """Number to Excel-style column name, e.g., 1 = A, 26 = Z, 27 = AA, 703 = AAA."""
    name = ''
    while n > 0:
        n, r = divmod (n - 1, 26)
        name = chr(r + ord('A')) + name
    return name

In [170]:
mapping_filename="check_file/mapping.csv"
raw_filename="data_source/Single Stock Total Return Raw Data.xlsm"
sheet_name='Sheet1'
# result_file_name

col_name=[]
col_idx=[]
with open(mapping_filename, 'r') as file:
    for line in file: 
        # Format XXX,col_name|col_idx
        temp = line.strip().split(',')
        col = temp[1].split('|')
        col_name.append(col[0])
        col_idx.append(col[1])

cols=','.join([excel_column_name(int(e) + 1) for e in col_idx])

print(col_name)
print(col_idx)
print(cols)


['ADVANCED INFO SER. ', 'AIRPORTS OF THAILAND ', 'ASIAN SEA CORPORATION ', 'ASIA AVIATION ', 'ASIA SERMKIJ LEASING ', 'BANGCHAK CORPORATION ORS ', 'BGK.AVIATION FUEL SVS. ', 'BANGKOK BANK ', 'BANGKOK BANK ', 'BANGKOK DUSIT MED.SVS. ', 'BANGKOK EXPRESSWAY AND METRO ', 'BANGKOK INSURANCE ', 'BANGKOK LIFE ASSURANCE ', 'BANK OF AYUDHYA ', 'BANK OF AYUDHYA ', 'AIRPORTS OF THAILAND ']
['9', '34', '45', '4', '47', '61', '55', '59', '59', '64', '67', '76', '78', '58', '58', '34']
J,AI,AT,E,AV,BJ,BD,BH,BH,BM,BP,BY,CA,BG,BG,AI


In [172]:
df = pd.read_excel(open(raw_filename, 'rb'),
#                    names=col_name,
                   usecols=cols,
                   sheet_name=sheet_name)
df.head()

,Unnamed: 4,Unnamed: 9,Unnamed: 34,Unnamed: 45,Unnamed: 47,Unnamed: 55,Unnamed: 58,Unnamed: 59,Unnamed: 61,Unnamed: 64,Unnamed: 67,Unnamed: 76,Unnamed: 78
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ASIA AVIATION - TOT RETURN IND,ADVANCED INFO SER. - TOT RETURN IND,AIRPORTS OF THAILAND - TOT RETURN IND,ASIAN SEA CORPORATION - TOT RETURN IND,ASIA SERMKIJ LEASING - TOT RETURN IND,BGK.AVIATION FUEL SVS. - TOT RETURN IND,BANK OF AYUDHYA - TOT RETURN IND,BANGKOK BANK - TOT RETURN IND,BANGCHAK CORPORATION ORS - TOT RETURN IND,BANGKOK DUSIT MED.SVS. - TOT RETURN IND,BANGKOK EXPRESSWAY AND METRO - TOT RETURN IND,BANGKOK INSURANCE - TOT RETURN IND,BANGKOK LIFE ASSURANCE - TOT RETURN IND
3,87190C(RI),314926(RI),28530Q(RI),140261(RI),31572P(RI),15382P(RI),749549(RI),933079(RI),140346(RI),314898(RI),8836T4(RI),755194(RI),68032M(RI)
4,NaN,7878.38,124.84,168.24,246.74,233.33,369.48,1368.55,62.34,5654.58,NaN,3974.47,314.66


In [173]:
df.shape

(2432, 13)

In [175]:
df = df[2:]
df

,Unnamed: 4,Unnamed: 9,Unnamed: 34,Unnamed: 45,Unnamed: 47,Unnamed: 55,Unnamed: 58,Unnamed: 59,Unnamed: 61,Unnamed: 64,Unnamed: 67,Unnamed: 76,Unnamed: 78
2,ASIA AVIATION - TOT RETURN IND,ADVANCED INFO SER. - TOT RETURN IND,AIRPORTS OF THAILAND - TOT RETURN IND,ASIAN SEA CORPORATION - TOT RETURN IND,ASIA SERMKIJ LEASING - TOT RETURN IND,BGK.AVIATION FUEL SVS. - TOT RETURN IND,BANK OF AYUDHYA - TOT RETURN IND,BANGKOK BANK - TOT RETURN IND,BANGCHAK CORPORATION ORS - TOT RETURN IND,BANGKOK DUSIT MED.SVS. - TOT RETURN IND,BANGKOK EXPRESSWAY AND METRO - TOT RETURN IND,BANGKOK INSURANCE - TOT RETURN IND,BANGKOK LIFE ASSURANCE - TOT RETURN IND
3,87190C(RI),314926(RI),28530Q(RI),140261(RI),31572P(RI),15382P(RI),749549(RI),933079(RI),140346(RI),314898(RI),8836T4(RI),755194(RI),68032M(RI)
4,NaN,7878.38,124.84,168.24,246.74,233.33,369.48,1368.55,62.34,5654.58,NaN,3974.47,314.66
5,NaN,7878.38,124.84,168.24,246.74,233.33,369.48,1368.55,62.34,5654.58,NaN,3974.47,314.66
6,NaN,7878.38,124.84,168.24,246.74,233.33,369.48,1368.55,62.34,5654.58,NaN,3974.47,314.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2427,78.51,14549.33,2101.53,1725.9,1002.53,899.57,729.07,1501.91,120.18,17857.93,168.54,9859.63,296.23
2428,78.51,14549.33,2101.53,1725.9,1002.53,899.57,729.07,1501.91,120.18,17857.93,168.54,9859.63,296.23
2429,76.65,14636.45,2093.54,1783.71,997.74,881.93,749.61,1489.7,123.66,17616.6,167.5,9859.63,290.7
2430,81.6,14767.13,2125.51,1874.54,1036.11,908.39,759.88,1520.22,123.16,17697.05,170.64,9896.42,287.93


In [176]:
result_file_name="check_file/result.csv"
df.to_csv(result_file_name, header=False, index=False)